In [38]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:22pt; line-height:40px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# [RAG 절차]

1. 문서를 읽는다
    %pip install --upgrade --quiet docx2txt
2. 문서를 쪼갠다
    %pip install -qU langchain-text-splitters
3. 쪼갠 문서를 임베딩하여 vector database에 넣음(local에 저장) cf. 클라우드에 저장
    %pip install -q langchain-chroma
4. 질문을 이용해 유사도 검색
5. 유사도 검색한 문서를 LLM에 질문과 함께 전달하여 답변 얻음(렝체인 사용 가능)
    %pip install -q langchain
    (https://smith.langchain.com에서 key생성 .env에 LANGCHAIN_API_KEY로 추가)

# 0. 패키지 설치

In [ ]:
# 문서읽어오기
%pip install --upgrade --quiet docx2txt

In [2]:
# 텍스트를 chunk로 나누는 기능만 있는 경량 모듈
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [3]:
# 벡터DB(로컬DB) 어제의 chromadb가 아님
%pip install -q langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [4]:
# langchain 사용
%pip install -q langchain

Note: you may need to restart the kernel to use updated packages.


# 1. 문서읽기(X)

In [5]:
%%time
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader("./data/소득세법(법률)(제21065호)(20260102).docx")
document = loader.load()

CPU times: total: 4.86 s
Wall time: 5.1 s


In [7]:
len(document)

1

# 2. 문서를 쪼개면서 읽기(O)
- https://docs.langchain.com/oss/python/integrations/splitters

## 2.1 1500토큰씩 쪼개서 읽어오기

In [8]:
import time
start = time.time()
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import TokenTextSplitter
loader = Docx2txtLoader('./data/소득세법(법률)(제21065호)(20260102).docx')
# gpt-4, gpt-4o, gpt-4 turbo, gpt4o-mini, embedding모델들은 다 같은 방식으로 토큰 추출
text_splitter = TokenTextSplitter(
    encoding_name="cl100k_base", #토큰을 세는 방식 이름
    chunk_size=1500,             # chunk 당 토큰 수 기준
    chunk_overlap=200
    # separators = ["\n", "\n\n"]파라미터가 없음
)
documents = loader.load_and_split(text_splitter=text_splitter)
runtime = time.time() - start
print("문서를 쪼개면서 읽는 시간 : ", runtime)

문서를 쪼개면서 읽는 시간 :  6.768277406692505


In [10]:
len(documents) # chunk 수

180

In [17]:
# chunk 글자수
# documents[0].page_content
print([len(document.page_content) for document in documents])

[1699, 1656, 1641, 1650, 1738, 1442, 1287, 1535, 1325, 1619, 1596, 1588, 1566, 1639, 1622, 1559, 1612, 1638, 1573, 1465, 1436, 1609, 1456, 1497, 1635, 1606, 1533, 1649, 1662, 1595, 1603, 1678, 1595, 1637, 1601, 1539, 1561, 1594, 1693, 1708, 1657, 1627, 1636, 1659, 1667, 1595, 1491, 1485, 1645, 1709, 1629, 1617, 1495, 1626, 1612, 1620, 1609, 1576, 1636, 1602, 1556, 1563, 1600, 1616, 1643, 1691, 1635, 1685, 1621, 1631, 1609, 1605, 1603, 1604, 1698, 1686, 1702, 1612, 1539, 1558, 1651, 2060, 1562, 1606, 1557, 1648, 1594, 1615, 1766, 1651, 1690, 1576, 1536, 1553, 1638, 1685, 1693, 1694, 1664, 1529, 1627, 1703, 1675, 1546, 1585, 1687, 1679, 1714, 1603, 1655, 1648, 1495, 1531, 1562, 1594, 1646, 1543, 1449, 1593, 1559, 1521, 1473, 1519, 1545, 1668, 1700, 1692, 1655, 1648, 1741, 1670, 1628, 1639, 1623, 1638, 1642, 1666, 1658, 1594, 1591, 1561, 1641, 1498, 1610, 1567, 1613, 1636, 1619, 1531, 1496, 1702, 1598, 1579, 1627, 1559, 1585, 1665, 1565, 1616, 1564, 1612, 1535, 1512, 1557, 1576, 1628, 165

In [22]:
# chunk 글자수 최대값, 최소값
print(max([len(document.page_content) for document in documents]))
print(min([len(document.page_content) for document in documents]))

2060
955


## 2.2 1500글자 쪼개서 읽어오기

In [2]:
import time
start = time.time()
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader("data/소득세법(법률)(제21065호)(20260102).docx")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 문서를 쪼갤 때 1500글자씩 chunking
    chunk_overlap=200,
    # separators=["\n\n", "\n", " ", ""]
)
# 재귀적으로 다음 순서대로 시도:
# 1. \n\n(문단 구분)
# 2. \n(줄바꿈)
# 3. " "(공백) 
# 4. "" - 최후에는 글자 단위로 chunking
documents = loader.load_and_split(text_splitter=text_splitter)
runtime = time.time() - start
print("문서를 1500글자로 쪼개면서 읽는 시간 :", runtime)
print("chunk 갯수 :", len(documents))

문서를 1500글자로 쪼개면서 읽는 시간 : 4.993933916091919
chunk 갯수 : 193


In [3]:
# chunk들의 글자수
print([len(document.page_content) for document in documents])

[1463, 1421, 1482, 1487, 1479, 1408, 1457, 1495, 1467, 1446, 1487, 1456, 1467, 1351, 1392, 1362, 1402, 1470, 1410, 1489, 1455, 1496, 1441, 1319, 1458, 1476, 1452, 1382, 1384, 1467, 1227, 1494, 1494, 1470, 1454, 1495, 1412, 1477, 1477, 1362, 1449, 1386, 1055, 1467, 1361, 1493, 1467, 1434, 1351, 1471, 1495, 1479, 1457, 1442, 1370, 873, 1419, 1357, 1353, 1316, 1349, 1452, 1439, 1363, 1433, 1412, 1306, 1200, 1411, 1452, 1421, 1318, 1416, 1333, 1308, 1385, 1479, 1495, 1399, 1375, 1360, 1353, 1382, 1446, 1356, 1409, 1483, 1486, 1157, 1233, 1443, 1474, 1369, 1439, 1451, 1495, 1443, 1489, 1484, 1407, 1432, 1436, 1468, 1442, 1477, 1396, 1423, 1282, 1496, 1486, 1376, 1342, 1466, 1385, 1491, 1477, 1470, 1385, 1477, 1445, 1485, 1373, 1495, 1443, 1419, 1456, 1451, 1305, 1454, 1411, 1443, 1488, 1404, 1419, 1339, 1451, 1288, 1450, 1481, 1419, 1369, 1479, 1480, 1461, 1414, 1419, 1463, 1481, 1486, 1387, 1485, 1448, 1367, 1364, 1391, 1446, 1414, 1414, 1414, 1473, 1417, 1474, 1419, 1342, 1406, 1338, 1138

In [7]:
print(max([len(document.page_content) for document in documents]))
print(min([len(document.page_content) for document in documents]))

1496
873


# 3. 쪼갠 문서를 임베딩 -> 벡터 베이터베이스 저장
- 임베딩 모델 : upstage의 solar-embedding-1-large-passage
- 벡터데이터베이스(벡터 store) : chroma

In [5]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
import os
load_dotenv()
embedding = UpstageEmbeddings(
    #api_key=os.getenv('UPSTAGE_API_KEY'),
    model="solar-embedding-1-large-passage"
) 

In [6]:
# embed_query() 한 문자열을 임베딩 벡터로 전환한 숫자 list를 return
len(embedding.embed_query("소득세법은 어쩌구"))

4096

In [7]:
embedding_vectors = embedding.embed_documents( # 여러 문자열을 임베딩 벡터로
    [
        documents[0].page_content,
        documents[1].page_content
    ]
)

In [8]:
print(len(embedding_vectors), len(embedding_vectors[0]), len(embedding_vectors[1]))
print(embedding_vectors[0][:10])

2 4096 4096
[0.007653872482478619, 0.004820186644792557, -0.01941058412194252, 0.003934051375836134, -0.01428203471004963, -0.0034666394349187613, -0.002564274473115802, -0.0016489258268848062, -0.01149054616689682, -0.0008731517009437084]


In [9]:
%%time
from langchain_chroma import Chroma
# 데이터 처음 저장할 때
database = Chroma.from_documents(
    documents=documents, # chunk
    embedding=embedding, # 임베딩 객체
    collection_name="tax-collection", # 생략시 이름 랜덤
    persist_directory="./chroma_upstage"  # 생략시 로컬DB에 저장 안 됨. 프로그램 종료시 DB날라감
)
# 이미 저장된 vector DB(store)를 사용할 때
# database = Chroma(
#     embedding_function=embedding,
#     collection_name="tax-collection",
#     persist_directory="./chroma"
# )

CPU times: total: 4.67 s
Wall time: 35.8 s


In [10]:
results = database._collection.get(include=['embeddings', 'documents', 'metadatas'])
print("데이터 수 :", len(results['ids']))
print("문서 임베딩 차원 수 :", len(results['embeddings'][0]))
print("1째 임베딩 샘플 :", results['embeddings'][1])
print("1번째 원본 :", results['documents'][1][:50])
print("1번째 metadata :", results['metadatas'][1])

데이터 수 : 193
문서 임베딩 차원 수 : 3072
1째 임베딩 샘플 : [ 0.01674929 -0.01080192 -0.0115323  ...  0.00672168 -0.01151033
 -0.00994524]
1번째 원본 : 1. 구성원 간 이익의 분배비율이 정하여져 있고 해당 구성원별로 이익의 분배비율이 확인되는
1번째 metadata : {'source': 'data/소득세법(법률)(제21065호)(20260102).docx'}


# 4. vector DB에 질문과 유사도 검색(답변 생성을 위한 retrieval)

In [13]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query=query,
                                           k=2) # 기본 k값은 4

In [14]:
#retrieved_docs

In [15]:
# print("\n\n---\n\n".join([doc.page_content for doc in retrieved_docs]))
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

# 5. 유사도검색으로 가져온 문서를 질문과 같이 LLM에 전달하여 답변 생성-1

In [25]:
from langchain_openai import ChatOpenAI
load_dotenv()
llm = ChatOpenAI(model="gpt-5-mini")

In [30]:
# prompt = f"""[identity]
# - 당신은 최고의 한국 소득세 전문가입니다
# - [context]를 참고해서 사용자의 질문에 답변해 주세요
# [context]는 다음과 같아요
# {retrieved_doc}
# 질문 : {query}"""
prompt = f"""너는 대한민국 세법(특히 소득세법)에 특화된 법령 분석 AI다.
반드시 아래 원칙을 따라 질문에 답변한다:
1. 제공된 문서(Context)에 근거한 내용만 답변한다.
2. 문서에 명시되지 않은 내용은 추론하거나 일반화하지 않는다.
3. 문서에서 근거를 찾을 수 없는 경우, [제공된 문서에는 해당 내용이 명시되어 있지 않습니다]라고 답한다.
[context]는 다음과 같다.
{retrieved_doc}
질문 : {query}"""

In [31]:
ai_message = llm.invoke(prompt)

In [28]:
ai_message.usage_metadata

{'input_tokens': 2063,
 'output_tokens': 1908,
 'total_tokens': 3971,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 1216}}

In [29]:
print(ai_message.content)

요청하신 계산은 제공된 문서에 근거해서만 답변해야 하므로, 먼저 문서에 명시된 사항과 누락된 사항을 정리합니다.

문서상 근거되는 내용(발췌)
- 제47조(근로소득공제)
  - 근로소득이 있는 거주자에 대해서 해당 과세기간에 받는 총급여액에서 “다음의 금액”을 공제한다. 다만 공제액이 2천만원을 초과하는 경우에는 2천만원을 공제한다.
  - 일용근로자 공제액은 1일 15만원.
  - 총급여액이 공제액에 미달하면 총급여액을 공제액으로 한다.
  - 2인 이상으로부터 근로소득을 받는 경우 합계 총급여액으로 계산한다.
- (참고로 문서에는 자녀세액공제 항목 일부와 연금계좌세액공제 규정도 포함되어 있음 — 예: 출산·입양 신고한 자녀의 경우 첫째 연 30만원, 둘째 연 50만원, 셋째 이상 연 70만원 등, 2명인 경우 연 55만원, 3명 이상인 경우 연 55만원＋초과 1인당 연 40만원 등)

하지만 계산에 필요한 핵심 자료 누락
- 제47조에서는 “다음의 금액”을 공제한다고 되어 있으나, 그 구체적 산식 또는 표(총급여액 구간별 공제액 등)가 문서에 포함되어 있지 않습니다.
- 소득세 과세표준별 세율표(소득세율)와 과세표준 산출 절차(근로소득에 대한 소득세 과세표준 계산 방법)도 문서에 제시되어 있지 않습니다.
- 기타 개인공제·인적공제·보험료·의료비·기부금 등 적용 여부와 지방소득세 등 계산에 필요한 규정도 문서에 없습니다.

결론
- 제공된 문서만으로는 “연봉 5천만원인 직장인의 소득세”를 계산할 수 없습니다.
- 따라서 [제공된 문서에는 해당 내용이 명시되어 있지 않습니다]

계산을 원하시면 문서에 없는 다음 정보가 필요합니다(문서에 근거하여 계산하려면 해당 항목들의 조문 또는 수치가 제공되어야 합니다).
- 근로소득공제의 구체적 산식 또는 표(총급여액 구간별 공제액)
- 소득세 과세표준별 세율표(과세표준 구간·세율·누진공제 등)
- 적용되는 인적공제·기타 소득공제 및 세액공제(예: 배우자·부양가족 여부, 연금계좌 납입액 등)

원하시면 위 항목들(근로소득공

# 5. 유사도검색으로 가져온 문서를 질문과 같이 LLM에 전달하여 답변 생성-2

In [82]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-nano")
promptTemplate = ChatPromptTemplate([
    ("system", "당신은 최고의 한국 소득세 전문가입니다"),
    ("human", f"""다음 문맥을 참고하여 질문에 답변하세요.
    답을 모르면 모른다고 말하세요.
    최대 3문장으로 간결하게 답변하세요.
    질문 : {{question}}
    문맥 : {{context}}
    답변 : """)
])
promptTemplate

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 최고의 한국 소득세 전문가입니다'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='다음 문맥을 참고하여 질문에 답변하세요.\n    답을 모르면 모른다고 말하세요.\n    최대 3문장으로 간결하게 답변하세요.\n    질문 : {question}\n    문맥 : {context}\n    답변 : '), additional_kwargs={})])

In [83]:
prompt = promptTemplate.invoke({
                            'context':retrieved_doc, # retrieved_docs보다 추천
                            'question':query
        })

In [84]:
llm.invoke(prompt)

AIMessage(content='연봉 5천만원인 직장인의 소득세는 개인의 공제, 세액공제 등에 따라 다르므로 정확한 계산이 필요합니다. 일반적으로 근로소득세율표와 기본공제, 보험료공제 등을 고려하면 약 700만원~1,000만원 정도의 세금이 부과될 수 있습니다. 정확한 세액은 구체적인 공제 항목과 소득 구성에 따라 달라지니, 세무 전문가와 상담하는 것을 권장드립니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 110, 'prompt_tokens': 2155, 'total_tokens': 2265, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7f8eb7d1f9', 'id': 'chatcmpl-Cvb7LH9aYjYXLjfq07OM5FDbpxlcC', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b9bad-ba1b-7833-8a26-ee9472f3d41f-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 2155, 'output_tokens': 110, 'total_tokens': 2265, 'input_token_details': {'audio': 0, 'cach

In [86]:
# 위의 예제를 한번에
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
output_parser.invoke(llm.invoke(promptTemplate.invoke({'context':retrieved_doc,
                                                       'question':query})))

'연봉 5천만원인 직장인의 소득세는 정확한 계산이 필요하지만, 대략적으로 근로소득세율 구간에 따라 10%에서 20% 수준입니다. 구체적인 세액은 각 공제 및 세율 적용 후 결정됩니다. 따라서, 상세 계산을 원하시면 추가 정보가 필요합니다.'

# 6. langchain으로 답변 생성

In [87]:
# 위의 예제를 langchain으로 답변생성
rag_chain = promptTemplate | llm | output_parser
rag_chain.invoke({'context':retrieved_doc,'question':query })

'연봉 5천만원인 직장인의 소득세는 구체적인 공제액과 세율에 따라 달라지며, 대략적인 소득세는 700만원 내외일 수 있습니다. 하지만 정확한 세액은 근로소득공제, 인적공제, 특별공제 등 각종 공제 후 계산되어야 합니다. 상세한 계산을 위해서는 공제 내역과 세법 적용이 필요합니다.'

## langchain 전달
    smith.langchain.com에서 key생성 후 .env에 LANGCHAIN_API_KEY 추가

In [32]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

# 1. LLM과 임베딩 초기화
load_dotenv()
# llm = ChatOpenAI(model = "gpt-4.1-mini")
from langchain_upstage import ChatUpstage
llm = ChatUpstage(model="solar-pro2")

embedding = UpstageEmbeddings(model="solar-embedding-1-large-passage")
# 2. vector store load
vectorstore = Chroma(
    embedding_function=embedding,
    collection_name="tax-collection",
    persist_directory="./chroma_upstage"
)
# 3. Retriever 생성
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":4})
# 4. 프롬프트 템플릿
template = f"""당신은 최고의 한국 소득세 전문가입니다.
다음 문맥을 참고하여 질문에 답하세요.
답을 모르면 모른다고 답하세요.
최대 3문장으로 간결하게 답변하세요.
질문 : {{query}}
문맥 : {{context}}
답변 : """
prompt = ChatPromptTemplate.from_template(template)
# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return  "\n\n---\n\n".join([doc.page_content for doc in documents])

In [33]:
# 6. RAG 체인 구성 (LCEL 방식)
from langchain_core.runnables import RunnablePassthrough # {"query":"~"} => "~"
rag_chain = (
    {
        "context":retriever | format_documents,
        "query":RunnablePassthrough() # 질문 그대로 전달
    }
    | prompt # prompt에 context와 query 변수 주입
    | llm    
    | StrOutputParser()
)
# 7. 실행
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
rag_chain.invoke(query)

'연봉 5천만원인 직장인의 소득세는 근로소득공제, 종합소득세율, 세액공제 등을 종합적으로 고려해야 하므로 단순 계산으로 답변 불가합니다. 제공된 문맥에는 근로소득 공제율 및 세율 정보가 없어 정확한 세액 산정이 불가능합니다. \n\n(참고: 근로소득 공제 후 과세표준에 6~45% 세율 적용, 추가 세액공제 가능)'